In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import dask.dataframe as dd

sns.set()

In [ ]:
feat_dict_cup = pd.read_excel('/kaggle/input/ab-it-cup/feat_dict_cup.xlsx', index_col='name_eng')

In [ ]:
feat_cup = pd.read_csv('/home/ivannewest/Downloads/alpha_it_cup/feat_cup.csv')
objects_cup = pd.read_csv('/home/ivannewest/Downloads/alpha_it_cup/objects_cup.csv')
feat_dict_cup = pd.read_excel('/home/ivannewest/Downloads/alpha_it_cup/feat_dict_cup.xlsx', index_col='name_eng')

In [ ]:
feat_cup = feat_cup[~feat_cup.drop(['client_id', 'dt'], axis=1).isna().all(axis=1)]

In [ ]:
feat_cup

Число владельцев карт

In [ ]:
all_owners = np.unique(objects_cup['owner_id'])

In [ ]:
all_owners.shape

In [ ]:
# 10 кандидатов на категориальные признаки
np.unique(feat_cup.dtypes, return_counts=True)

In [ ]:
cat_vals = feat_cup.dtypes[feat_cup.dtypes == object].index
num_vals = feat_cup.dtypes[feat_cup.dtypes == float].index
cat_vals

In [ ]:
feat_dict_cup.loc[cat_vals]

In [ ]:
feat_cup[cat_vals].head()

In [ ]:
feat_dict_cup.loc[num_vals]['name_rus'].to_list()

In [ ]:
cat_vals = cat_vals.drop('client_id')

# Исследуем категориальные признаки

Теперь исследуем каждую категориальную фичу по отдельности на возможные значения

1. dt

In [ ]:
np.unique(feat_cup['dt'])

In [ ]:
len(np.unique(feat_cup['dt'])) # Всего 26 различных значений

2. country

In [ ]:
np.unique(feat_cup['country'].astype('str'))

In [ ]:
len(np.unique(feat_cup['country'].astype('str'))) # всего 73 различных значения

In [ ]:
sum(feat_cup['country'].isna()) # число пропусков

3. clientoutflowstatus

In [ ]:
np.unique(feat_cup['clientoutflowstatus'].astype(str))

In [ ]:
len(np.unique(feat_cup['clientoutflowstatus'].astype(str))) # Всего 10 различных значений

In [ ]:
sum(feat_cup['clientoutflowstatus'].isna()) # число пропусков

4. stratsegfactor(Причина попадания в сегмент)

In [ ]:
np.unique(feat_cup['stratsegfactor'].astype(str))

In [ ]:
len(np.unique(feat_cup['stratsegfactor'].astype(str)))

In [ ]:
sum(feat_cup['stratsegfactor'].isna()) # число пропусков

5. srvpackage(Пакет услуг клиента на отчетную дату)

In [ ]:
np.unique(feat_cup['srvpackage'].astype(str))

In [ ]:
len(np.unique(feat_cup['srvpackage'].astype(str)))

In [ ]:
sum(feat_cup['srvpackage'].isna()) # число пропусков

6.clientsegment(Сегмент клиента)

In [ ]:
np.unique(feat_cup['clientsegment'].astype(str))

In [ ]:
len(np.unique(feat_cup['clientsegment'].astype(str)))

In [ ]:
sum(feat_cup['clientsegment'].isna()) # число пропусков

7.gender

In [ ]:
np.unique(feat_cup['gender'].astype(str))

In [ ]:
sum(feat_cup['gender'].isna()) # число пропусков

8. addrref(Регион отделения)

In [ ]:
np.unique(feat_cup['addrref'].astype(str))

In [ ]:
len(np.unique(feat_cup['addrref'].astype(str)))

In [ ]:
sum(feat_cup['addrref'].isna()) # число пропусков

9. hdb_relend_client_credits_count_grade('a - только один кредит наличными, b - 1 к.н. + <= 2 крединых карт, c - только более одного к.н., d - остальное')

In [ ]:
np.unique(feat_cup['hdb_relend_client_credits_count_grade'].astype(str))

In [ ]:
sum(feat_cup['hdb_relend_client_credits_count_grade'].isna()) # число пропусков

Краткий вывод:
1. Больше всего пропусков у фичи hdb_relend_client_credits_count_grade(109305) это практически половина, при этом как мне кажется она достаточно важна
2. стоит сбросить addrref(адрес отделения) не думаю, что эта фича несет что-либо полезное

In [ ]:
feat_cup_dd = dd.read_csv('/kaggle/input/ab-it-cup/feat_cup.csv')
objects_cup_dd = dd.read_csv('/kaggle/input/ab-it-cup/objects_cup.csv')

In [ ]:
cat_vals = feat_cup_dd.dtypes[feat_cup_dd.dtypes == object].index
num_vals = feat_cup_dd.dtypes[feat_cup_dd.dtypes == float].index
cat_vals

In [ ]:
feat_cup_dd['age'].sum().compute()

# Ислледование корреляции признаков

In [ ]:
corr_matrix = feat_cup_dd.corr().compute()

In [ ]:
corr_matrix.to_csv('corr_matrix.csv')

Самая низкая корреляция

In [ ]:
np.sort(np.sort(corr_matrix.values, -1)[:, 0])

Самая высокая корреляция

In [ ]:
np.sort(np.sort(corr_matrix.values, -1)[:, ::-1][:, 1])[::-1]

In [ ]:
np.triu(np.ones(corr_matrix.shape),k=1)

In [ ]:
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))
upper_tri 

In [ ]:
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.85)]
print(to_drop)

In [ ]:
len(to_drop)

# Число пустых значений у каждого признака

In [ ]:
feat_cup_withouts_na = feat_cup_dd[~feat_cup_dd.drop(['client_id', 'dt'], axis=1).isna().all(axis=1)] #сначала удалим просто пустые значения 

In [ ]:
feat_cup_withouts_na.compute().shape

In [ ]:
amount_of_nans = feat_cup_withouts_na.isna().sum().compute()

In [ ]:
np.sort(amount_of_nans)[::-1]

In [ ]:
feat_dict_cup.loc[np.argsort(amount_of_nans)[::-1].index]['name_rus'].values

# Исследовал данные на дупликаты строк

In [ ]:
feat_cup_withouts_na = feat_cup_dd[~feat_cup_dd.drop(['client_id', 'dt'], axis=1).isna().all(axis=1)]

In [ ]:
feat_cup_without_duplicates2 = feat_cup_dd.drop_duplicates().compute()

In [ ]:
feat_cup_without_duplicates3 = feat_cup_withouts_na.drop(['client_id'], axis=1).drop_duplicates().compute()

In [ ]:
duplicates_by_data = feat_cup_withouts_na.drop(['dt'], axis=1).compute().duplicated()

In [ ]:
sum(duplicates_by_data)

In [ ]:
duplicates_by_data2 = feat_cup_withouts_na.drop(['client_id'], axis=1).compute().duplicated()

In [ ]:
sum(duplicates_by_data2)

In [ ]:
duplicates_by_data3= feat_cup_withouts_na.drop(['client_id', 'dt'], axis=1).compute().duplicated()

In [ ]:
sum(duplicates_by_data3)

In [ ]:
feat_cup_without_duplicates3.shape 

In [ ]:
objects_cup_dd

# Исследуем на дупликаты колонок

In [ ]:
feat_cup_enc = feat_cup_withouts_na.copy().compute()

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
for col in tqdm(feat_cup_enc.columns):
    feat_cup_enc[col] = feat_cup_enc[col].factorize()[0]

In [ ]:
dup_cols = {}

for i, c1 in enumerate(tqdm(feat_cup_enc.columns)):
    for c2 in feat_cup_enc.columns[i + 1:]:
        if c2 not in dup_cols and np.all(feat_cup_enc[c1] == feat_cup_enc[c2]):
            dup_cols[c2] = c1

In [ ]:
dup_cols 

Дублирующихся колонок нет

# Гистограмма различных значений у признаков

In [ ]:
nunique = feat_cup_withouts_na.nunique().compute()

In [ ]:
n = feat_cup_withouts_na.compute().shape[0]

In [ ]:
plt.figure(figsize=(14, 6))
_ = plt.hist(nunique.astype(float)/n, bins=250)

In [ ]:
mask = (nunique.astype(float)/n > 0.7)

In [ ]:
feat_cup_withouts_na.loc[:, mask].compute()

In [ ]:
feat_cup_withouts_na[mask.index[mask]].describe().compute()

In [ ]:
feat_dict_cup.loc[mask.index[mask]]

In [ ]:
plt.hist(feat_cup_withouts_na['avg_amount_daily_transactions_90d'].compute(), bins=100)